In [ ]:
Autocad_File_Integration = "On"

from pyautocad import Autocad, APoint
import math 
import pandas as pd 
import matplotlib.pyplot as plt
import os

def tonnage(Number_bars,diameter_mm,Length_m): 
    """
    Function that returns the tonnage of rebar using the knowledge of how many bars there are per tonne stored in dictiionaries 
    
    Args: 
    Number_bars(int): The number of rebars 
    diameter_mm(int): The diameter of the rebar in mm
    Length_m(float): The standard length of the rebar in meters

    Returns: 
    float: The Tonnage of the Rebar
    """
    Bar_per_tonne_6m = { 
    10: 270,
    12:188,
    16:106,
    20:68,
    25:43,
    32:26

    }
    Bar_per_tonne_7_5m = { 
    10: 216,
    12:150,
    16:85,
    20:54,
    25:35,
    32:21

    }
    Bar_per_tonne_9m = { 
    10: 180,
    12:125,
    16:70,
    20:45,
    25:29,
    32:18

    }
    Bar_per_tonne_12m = { 
    10: 135,
    12:94,
    16:53,
    20:34,
    25:22,
    32:13

    }

    
    if  Length_m == "6m":
        return Number_bars / Bar_per_tonne_6m[diameter_mm]
    elif Length_m == "7.5m":
        return Number_bars / Bar_per_tonne_7_5m[diameter_mm]
    elif Length_m == "9m":
        return Number_bars / Bar_per_tonne_9m[diameter_mm]
    elif Length_m == "12m":
        return Number_bars / Bar_per_tonne_12m[diameter_mm]
    else:
        return "Invalid length. Maybe use the Linear  metre option?" 

def bars_lengths(tonnage,Length_m,diameter):
    """
    Function that returns the number of bars using the knowledge of how many bars there are per tonne stored in dictiionaries 
    
    Args: 
    Tonnage(float): The number of rebars 
    diameter_mm(int): The diameter of the rebar in mm
    Length_m(float): The standard length of the rebar in meters

    Returns: 
    float: The number of bars in the tonnage of rebar
    """
    
    Bar_per_tonne_6m = { 
    10: 270,
    12:188,
    16:106,
    20:68,
    25:43,
    32:26

    }
    Bar_per_tonne_7_5m = { 
    10: 216,
    12:150,
    16:85,
    20:54,
    25:35,
    32:21

    }
    Bar_per_tonne_9m = { 
    10: 180,
    12:125,
    16:70,
    20:45,
    25:29,
    32:18

    }
    Bar_per_tonne_12m = { 
    10: 135,
    12:94,
    16:53,
    20:34,
    25:22,
    32:13

    }
    if  Length_m == "6m":
        return tonnage * Bar_per_tonne_6m[diameter]
    elif Length_m == "7.5m":
        return tonnage* Bar_per_tonne_7_5m[diameter]
    elif Length_m == "9m":
        return tonnage * Bar_per_tonne_9m[diameter]
    elif Length_m == "12m":
        return tonnage * Bar_per_tonne_12m[diameter]
    else:
        return "Invalid length. Maybe use the Linear  metre option?"

def Cutlength(lengths,diameter,number_90_bends): 
    """
    Calculates the Cutlength when given a list of measurements. All meaasurements are in mm 

    Args: 
    lengths (list): A List of Edge to Edge measurements of a piece of rebar in mm 
    diameter(int):  The diameter of the rebar in mm
    number_90_bends (int): number of 90 degree bend equivalent to account for bending deductions 
    show(str): Flag indicating whether to show the Cutlength
    Barmark (str): Barmark Label

    Returns: 
    float: The Cutlength of a linear piece of rebar

    
    """
    sum_lengths = sum(lengths)
    Bend_deductions= { 
        10:20,
        12:24,
        16:32,
        18:36,
        20:40,
        25:50 }
    bend_deduction = Bend_deductions[diameter] * number_90_bends
    Cutlength = sum_lengths - bend_deduction

    return Cutlength    

def numof(length,spacing,cover): 
   
        num_of_elements = math.ceil((length-cover)/spacing) -2
        return num_of_elements
def stirrup_cutting_length(Perimeter, bar_diameter):
    """ Calculates the Cutting Length of a stirrup or similar ligature. All measurements in mm """
    # Inset Secondary Function that calculates the perimeter
    
    # Hook length (assuming 9d for each hook)
    hook_length = 2 * bar_diameter
    
    # Bend length (assuming 3 bends at 90° and 2 bends at 135°)
    bend_length = 3 * 2 * bar_diameter + 2 * 3 * bar_diameter
    
    # Cutting length
    cutting_length = Perimeter + hook_length - bend_length
    
    return cutting_length

def p_square(length): 
    return 4*length 
def p_rectangle(length,width): 
    return 2*length + 2*width 
def p_circle(diameter): 

    return math.pi * 2 * diameter
def bars_and_offcuts(cut_length, bar_size, num_cuts_needed):
    N_t = num_cuts_needed
    N_c = bar_size // cut_length
    B_s = bar_size
    C_L = cut_length

    if N_t <= N_c:
        N_u = 1
        O_s1 = B_s - (C_L * N_t)
        return {
            "Number of Bars used": N_u,
            "Offcut 1 Size": O_s1
        }
    else:
        N_u = math.floor(N_t / N_c)
        N_r = N_t - (N_u * N_c)
        O_s2 = B_s - (C_L * N_r)
        O_s1 = B_s - (C_L * N_c)
        copies_of_offcut_1 = N_u

        if O_s2 >= B_s:
            return {
                "Number of Bars used": copies_of_offcut_1,
                "Offcut 1 Size": O_s1,
                "Offcut 2 Size": 0,
                "Copies of Offcut 1": copies_of_offcut_1
            }
        else:
            return {
                "Number of Bars used": N_u + 1,
                "Offcut 1 Size": O_s1,
                "Offcut 2 Size": O_s2,
                "Copies of Offcut 1": copies_of_offcut_1
            }
def bars_and_offcuts(cut_length, bar_size, num_cuts_needed):
    N_t = num_cuts_needed
    N_c = bar_size // cut_length
    B_s = bar_size
    C_L = cut_length

    if N_t <= N_c:
        N_u = 1
        O_s1 = B_s - (C_L * N_t)
        return {
            "Number of Bars used": N_u,
            "Offcut 1 Size": O_s1
        }
    else:
        N_u = math.floor(N_t / N_c)
        N_r = N_t - (N_u * N_c)
        O_s2 = B_s - (C_L * N_r)
        O_s1 = B_s - (C_L * N_c)
        copies_of_offcut_1 = N_u

        if O_s2 >= B_s:
            return {
                "Number of Bars used": copies_of_offcut_1,
                "Offcut 1 Size": O_s1,
                "Offcut 2 Size": 0,
                "Copies of Offcut 1": copies_of_offcut_1
            }
        else:
            return {
                "Number of Bars used": N_u + 1,
                "Offcut 1 Size": O_s1,
                "Offcut 2 Size": O_s2,
                "Copies of Offcut 1": copies_of_offcut_1
            }
def optimal_bar_size(cut_length, num_cuts_needed, p):
    standard_bar_sizes = [6, 7.5, 9, 12]
    min_offcut_sum = float('inf')
    optimal_bar_size = None
    optimal_bars_required = 0
    offcut_sums = []

    if cut_length > 12:
        optimal_bar_size = 12
        result = bars_and_offcuts(12, optimal_bar_size, num_cuts_needed)
        optimal_bars_required = result.get("Number of Bars used", 0)
        print(f"Warning: Cut length {cut_length} is greater than 12. Using bar size 12.")
        return optimal_bar_size, optimal_bars_required

    for bar in standard_bar_sizes:
        if bar < cut_length:
            continue

        result = bars_and_offcuts(cut_length, bar, num_cuts_needed)
        offcut_sum = result.get("Offcut 1 Size", 0) * result.get("Copies of Offcut 1", 0) + result.get("Offcut 2 Size", 0)
        offcut_sums.append(offcut_sum)

        if offcut_sum < min_offcut_sum:
            min_offcut_sum = offcut_sum
            optimal_bar_size = bar
            optimal_bars_required = result.get("Number of Bars used", 0)
    
    if p == "Plot":
        plot_offcut_sums(standard_bar_sizes, cut_length, offcut_sums)
        return optimal_bar_size, optimal_bars_required
    else:
        return optimal_bar_size, optimal_bars_required
def bm(Barmark,Lengths,Type,Diameter,bends_90,Unit_number,Location):
    
    import pandas as pd
    CutL = Cutlength(Lengths,Diameter,bends_90)/1000
    Number_units = Unit_number
    Grade = str(Type) + str(Diameter)
    Forecasted = "No"
    if Forecasted == "Yes":
        Preferred_Length = optimal_bar_size(CutL,Number_units,"No")
    else:
        x = bars_and_offcuts(CutL,6,Number_units)
        Preferred_Length = 6
        Preferred_Length_used = x["Number of Bars used"]
    
    My_Bar = { 
    "Barmark":[Barmark],
    "Grade":[Grade],
    "Location":[Location],
    "Cutlength": [CutL],
    "Number of Units": [Number_units],
    "Preferred Length": [Preferred_Length],
    "Preferred_Length_Used":[Preferred_Length_used],
    "Lengths":[Lengths]
    
    }
    
    My_Bar = pd.DataFrame(My_Bar)
    return My_Bar

## AutoCad Integration Code Starts Here

def get_table_info():
    table_info = []

    # Iterate over all entities in the active document
    for entity in acad.iter_objects('Table'):
        handle = entity.Handle
        width = entity.Columns
        height = entity.Rows
        table_info.append((handle, width, height))

    return table_info


if Autocad_File_Integration == "On":
    # Initialize AutoCAD COM server
    acad = Autocad(create_if_not_exists=True)
    # Retrieve table info
    table_info = get_table_info()
    for handle, width, height in table_info:
        print(f"Handle: {handle}, Columns: {width}, Rows: {height}")


def insert_df_into_table(table_handle, df, start_row):

    """
    Inserts a DataFrame into an AutoCAD table starting from a specified row.

    Args:
        table_handle (str): The handle of the AutoCAD table.
        df (pd.DataFrame): The DataFrame to insert.
        start_row (int): The row number to start inserting data from (1-indexed).
    """

    # Iterate over all entities in the active document to find the table
    for entity in acad.iter_objects('Table'):
        if entity.Handle == table_handle:
            table = entity
            break
    else:
        print(f"Table with handle {table_handle} not found.")
        return

    # Get the number of columns in the table
    num_cols = table.Columns

    # Get the number of rows in the table
    num_rows = table.Rows

    # Iterate over rows of the DataFrame
    for i, row in enumerate(df.values.tolist()):
        # Calculate the target row in the table
        target_row = start_row + i

        # Check if the target row is within the table bounds
        if target_row > num_rows:
            break

        # Iterate over columns of the DataFrame
        for j, cell_value in enumerate(row):
            # Check if the column index is within the table bounds
            if j >= num_cols:
                break

            # Insert data into the table
            table.SetText(target_row, j + 1, str(cell_value))
HD = "HD"
def Steel_weight(diameter,length): 
    Weight = ((diameter**2)/162) * length
    return round(Weight,2)

def extract_last_two_as_float(value):
    try:
        # Extract last two characters
        last_two_chars = value[-2:]
        # Convert to float
        return float(last_two_chars)
    except ValueError:
        # Handle the case where conversion fails
        return None  # or you can return a default value like 0.0

def Order_Details(panel):
    pivot_table_lengths = pd.pivot_table(panel, values='Preferred_Length_Used',
    index=['Grade'], columns='Preferred Length',aggfunc ='sum')

    panel['diameter'] = panel["Grade"].apply(extract_last_two_as_float)
    pivot_table_lengths

    panel["Weight Used"] = panel.apply(lambda x: Steel_weight(x['diameter'],x['Cutlength']),axis =1)
    panel["Weight Used"] = panel['Weight Used']*panel['Number of Units']
    panel['LM Ordered'] = panel['Preferred Length']*panel['Preferred_Length_Used']
    panel['Weight Ordered'] = panel.apply(lambda x:Steel_weight(x['diameter'],x['LM Ordered']),axis =1)

    pivot_table_weight = pd.pivot_table(panel,values = "Weight Ordered",
        columns = 'Preferred Length',index = 'Grade',aggfunc = 'sum')

    print(pivot_table_weight)
    print("")
    print(pivot_table_lengths)

def Lapped_bars(standard_length,diameter,Lapping_distance): 
    """
    Returns the Number of Bars that have to be lapped in a span

    Args: 
    stardard_length(int): The stardard length of the bar to be used for lapping in mm 
    diameter: The diameter of the bar in mm for lap length calculation 
    Lapping_distance: The distance that has to be covered using laps 

    Returns: 

    number_of_bars(int): The total number of bars in the lapping sequence 
    length_left(int): The length of the non-crank stright bar at the end 
    
    """
    lap_length = 60 * diameter
    Bar_less_lap = standard_length - lap_length
    barby2span = Bar_less_lap*2 + lap_length
    Bars_double_lap = math.floor(Lapping_distance/barby2span)
    Length_left = Lapping_distance - Bars_double_lap*barby2span
    number_of_bars = 2*Bars_double_lap

    
    if Length_left>= standard_length:
        Length_left =  Length_left - standard_length 
        number_of_bars = number_of_bars + 2
        print(f"Total Number of Bars Used = {number_of_bars} with a straight bar of {Length_left} added at the end after {number_of_bars-1} cranked and lapped")
        return number_of_bars, Length_left
    else:
        print(f"Length left after lapping = {Length_left} and number of bars used = {number_of_bars}")
        return number_of_bars, Length_left
    